In [1]:
### Environment setup
from pystac_client import Client
import planetary_computer as pc
import os

# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)
env_vars = !cat /content/.env

for var in env_vars:
    key, value = var.split(' = ')
    os.environ[key] = value

In [2]:
import pandas as pd
import fsspec
import numpy as np
import geopandas as gpd
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
#experiments with dask
import dask.dataframe as dd

container = 'itv-data'

storage_options={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY']}
fs = fsspec.filesystem('az', account_name=storage_options['account_name'], account_key=storage_options['account_key'])
station_url = f'az://{container}/itv_station_metadata.csv'
station_df = dd.read_csv(station_url, storage_options=storage_options)                
#df = dd.read_csv(station_url, storage_options=storage_options)
#ddf = dg.from_dask_dataframe(df).set_geometry(dg.points_from_xy(ddf, 'Latitude', 'Longitude'))

In [3]:
station_df.head()

,site_no,station_name,Latitude,Longitude,geometry
0,ITV1,Catete PA-279,-6.763,-50.871,POINT (-50.871 -6.763)
1,ITV2,Catete MOP,-6.529,-51.059,POINT (-51.059 -6.529)
2,ITV3,Tapirape,-5.669,-50.307,POINT (-50.307 -5.669)
3,ITV4,Parauapebas Sossego,-6.441,-50.035,POINT (-50.035 -6.441)
4,ITV5,Parauapebas Est Captacao,-6.092,-49.907,POINT (-49.907 -6.092)


# Here we preprocessing of the data for standardized ingestion

In [5]:
site_df

,"Campaign,Campaign Date,Drainage Area (km²),SSC (mg/l),Q (m³/s)"
0,"1,24/03/2015,376.29,24.93,40.59"
1,"2,26/05/2015,376.29,15.61,8.21"
2,"3,18/09/2015,376.29,12.94,2.24"
3,"4,16/02/2016,376.29,16.29,4.17"
4,"5,12/04/2016,376.29,21.65,10.17"
5,"6,21/06/2016,376.29,15.52,3.01"
6,"7,13/09/2016,376.29,14.11,1.21"
7,"8,31/01/2017,376.29,23.61,8.09"
8,"9,04/04/2017,376.29,21.07,16.02"
9,"10,20/06/2017,376.29,12.15,4.58"


In [8]:
for site_no in station_df.site_no:
    station_url = f'az://{container}/{site_no}.csv'
    site_df = pd.read_csv(station_url,\
                          storage_options=storage_options,\
                          delimiter=',')

    site_df['Date-Time']=pd.to_datetime(site_df['Campaign Date'], \
                                       format='%d/%m/%Y')

    write_filename = f'az://{container}/stations/{site_no}.csv'
    site_df.to_csv(write_filename, storage_options=storage_options,\
                  index=False)